In [10]:
import PySimpleGUI as sg
from ToDoQueue.queue.task import Task
from ToDoQueue.queue_manager import ToDo

In [11]:
sample = Task(name='test',description='Task for testing Gui with', est_mins_to_finish=10, priority=False, link='')
sample2 = Task(name='test2',description='Another Task for testing Gui with', est_mins_to_finish=8, priority=False, link='')

In [12]:


current_task = sample

In [13]:
def update_window(window:sg.Window, new_task:Task) -> sg.Window:
    window['-MIN_NAME-'].update(new_task.name)
    window['-MIN_ETC-'].update(new_task.est_mins_to_finish)
    window['-MAX_NAME-'].update(new_task.name)
    window['-MAX_ETC-'].update(new_task.est_mins_to_finish)
    window['-MAX_LINK-'].update(new_task.rel_link)
    window['-MAX_DESCR-'].update(new_task.description)
    current_task = new_task
    return window

In [14]:
FORM_DEFAULTS = {
    '-NEW_NAME-':'New Task Name',
    '-NEW_PRIO-':'Priority Item?',
    '-NEW_DESCR-':'New Task Description',
    '-NEW_ETC-':'Estimated Time to Complete Task',
    '-NEW_LINK-':'New Task Link',
}


In [15]:
def clear_new_task(window:sg.Window) -> sg.Window:
    for key in FORM_DEFAULTS.keys():
        if key == '-NEW_PRIO-':
            window[key].update(False)
        else:
            window[key].update(FORM_DEFAULTS[key])
    return window

In [16]:
new_task_layout = [[sg.Text('New Task'), 
                    sg.InputText(key="-NEW_NAME-",default_text=FORM_DEFAULTS["-NEW_NAME-"]), sg.Checkbox(key="-NEW_PRIO-", text=FORM_DEFAULTS["-NEW_PRIO-"], default=False)], # Line 1
                    [sg.InputText(key="-NEW_LINK-",default_text=FORM_DEFAULTS["-NEW_LINK-"]), 
                    sg.InputText(key="-NEW_ETC-",default_text=FORM_DEFAULTS["-NEW_ETC-"])], # Line 2
                    [sg.Multiline(key="-NEW_DESCR-",default_text=FORM_DEFAULTS["-NEW_DESCR-"])], # Line 3
                    [sg.Button('Submit')]]

cur_task_min_layout = [[sg.Text('Current Task Min')],
                        [sg.Text(f"{current_task.name}", key='-MIN_NAME-'), sg.Text(f"{current_task.est_mins_to_finish}", key='-MIN_ETC-')],
                        [sg.Button('Max'), sg.Button('Complete'), sg.Button('Exit')]]

cur_task_detail_layout = [[sg.Text('Current Task Details')],
                            [sg.Text(f"{current_task.name}", key='-MAX_NAME-'), sg.Text(f"{current_task.est_mins_to_finish}", key='-MAX_ETC-')],
                            [sg.Text(f"{current_task.rel_link}", key='-MAX_LINK-')],
                            [sg.Text(f"{current_task.description}", key='-MAX_DESCR-')],
                            [sg.Button('Min'), sg.Button('New'), sg.Button('Complete'), sg.Button('Exit')]]

completed_task_layout = [[sg.Text('Complete Task')],
                        [sg.InputText(key="-COM_NOTES-" ,default_text='Completion Notes')],
                        [sg.Button('Close'), sg.Button('Freeze')]]

In [17]:
Layout = [[sg.Column(cur_task_min_layout, key='-CUR_TASK_MIN-'),
             sg.Column(new_task_layout, key='-NEW_TASK-', visible=False), 
            sg.Column(cur_task_detail_layout, key='-CUR_TASK_DETAIL-', visible=False),
            sg.Column(completed_task_layout, key='-COMPLETE-', visible=False)]]

In [18]:
window = sg.Window('Swapping the contents of a window', Layout)

layout = "-CUR_TASK_MIN-"  # The currently visible cur_layout
while True:
    event, values = window.read()
    print(event, values)
    if event in (None, 'Exit') or 'Exit' in event:
        break
    if event == 'Max': # Shift from min view to max, detailed view
        window[layout].update(visible=False)
        window[f'-CUR_TASK_DETAIL-'].update(visible=True)
        layout = '-CUR_TASK_DETAIL-'
    elif event == 'Min': # Shift from Maximum detailed view, to minimum focused view
        window[layout].update(visible=False)
        window[f'-CUR_TASK_MIN-'].update(visible=True)
        layout = '-CUR_TASK_MIN-'
    elif event == 'New': # Add a new task
        window[layout].update(visible=False)
        window['-NEW_TASK-'].update(visible=True)
        layout = '-NEW_TASK-'
    elif 'Submit' in event: # Submit newly created task and add it to the queue
        window[layout].update(visible=False)
        # Create new Task with input data, add to queue
        window = clear_new_task(window)
        window['-CUR_TASK_MIN-'].update(visible=True)
        layout = '-CUR_TASK_MIN-'
    elif 'Complete' in event:  # Complete the current task. Mark it complete, and leave some notes
        window[layout].update(visible=False)
        window['-COMPLETE-'].update(visible=True)
        layout = '-COMPLETE-'
    elif event == 'Freeze': # Freeze task; put on hold temporarily. Will be re-added to front of queue, and become the current task again after the new current task is completed. 
        pass                # Functionality and usage still under development. 
    elif event == 'Close': # Close the current task. File it under completed, and get the next task.
        window[layout].update(visible=False)
        # Get next Task, set current_task = next Task
        window = update_window(window, current_task)
        window['-CUR_TASK_MIN-'].update(visible=True)
        layout = '-CUR_TASK_MIN-'



window.close()

None {'-NEW_NAME-': None, '-NEW_PRIO-': None, '-NEW_LINK-': None, '-NEW_ETC-': None, '-NEW_DESCR-': None, '-COM_NOTES-': None}
